In [1]:
import os
import sys
sys.path.append("/home/pervinco/Upstage_Ai_Lab/Final/IR/src")

import logging

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

import json
import random
import huggingface_hub

from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

voyage_api_key = os.getenv('VOYAGE_API_KEY')
os.environ['VOYAGE_API_KEY'] = voyage_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

from langchain.schema import Document
from langchain_community.vectorstores.faiss import FAISS

from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_teddynote.retrievers import EnsembleRetriever, EnsembleMethod

from config import Args
from data.data import load_document, chunking, load_eval
from sparse_retriever.model import load_sparse_model
from dense_retriever.model import load_dense_model, load_upstage_encoder, load_hf_encoder

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


/home/pervinco/miniconda3/envs/ir-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/pervinco/.config/sagemaker/config.yaml


In [2]:
args = Args()
args.eval_file_path = "../dataset/processed_eval.jsonl"
args.doc_file_path = "../dataset/gpt_contextual_retrieval_documents_v3.jsonl"
args.faiss_index_file = "../src/index_files/upstage/CRV3_2"

documents = load_document(args.doc_file_path)
print(len(documents))

evals = load_eval(args.eval_file_path)
print(len(evals))

24799
220


In [3]:
sparse_retriever = load_sparse_model(documents, args.src_lang)
sparse_retriever.k = 20

dense_retriever = load_dense_model(args, documents).as_retriever(search_kwargs={"k": 20})

if args.ensemble_method == "rrf":
    retriever = EnsembleRetriever(retrievers=[sparse_retriever, dense_retriever], method=EnsembleMethod.RRF)

elif args.ensemble_method == "cc":
    retriever = EnsembleRetriever(retrievers=[sparse_retriever, dense_retriever], method=EnsembleMethod.CC)


FAISS 인덱스 로드 중: ../src/index_files/upstage/CRV3_2
FAISS 인덱스 로드 완료, 총 문서 수: 24799


In [4]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=retriever, llm=llm)

/tmp/ipykernel_2007317/1965889648.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


In [5]:
query = evals[0]['query']
unique_docs = retriever_from_llm.get_relevant_documents(query=query)
print(len(unique_docs))

/tmp/ipykernel_2007317/104632218.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  unique_docs = retriever_from_llm.get_relevant_documents(query=query)
INFO:langchain.retrievers.multi_query:Generated queries: ['나무의 분류 체계를 연구하는 방법은 무엇인가요?  ', '나무를 분류하는 다양한 방법에 대해 알아보려면 어떻게 해야 하나요?  ', '나무의 분류에 관한 정보를 찾기 위한 효과적인 조사 방법은 무엇인가요?']


69


In [6]:
for doc in unique_docs:
    print(f"{doc.metadata['docid']}\n{doc.metadata['score']}\n{doc.page_content}")

a3071404-122f-408b-86a0-1787b4449a1b
1.0
기존의 분류 체계를 개선할 수 있습니다. 이러한 연구는 생물 다양성을 이해하고 보존하는 데에도 큰 도움이 됩니다. 과학자들은 이러한 연구를 통해 우리가 아직 알지 못하는

이 청크는 생물학적 연구와 관련된 문서에서 발췌되었습니다. 이 문서에서는 과학자들이 생물체를 분류하고 연구하는 과정에서 기존의 분류 체계를 개선할 수 있는 방법을 설명하고 있습니다. 또한, 이러한 연구가 생물 다양성을 이해하고 보존하는 데 기여하는 중요성을 강조하고 있으며, 아직 밝혀지지 않은 생물체들의 비밀을 해독하는 데에 대한 과학자들의 역할을 다루고 있습니다.
c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d
0.9186683706301946
크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은

이 청크는 생물 분류학과 관련된 연구 문서에서 발췌되었습니다. 이 문서에서는 한 학생이 나무의 성장 속도와 온도 범위가 비슷한 두 나무를 조사하면서, 잎과 꽃의 차이를 통해 이 나무들이 같은 속에 속할 가능성을 추측하는 과정을 설명하고 있습니다. 이 설명은 나무의 분류와 관련된 중요한 정보를 제공하며, 생물 분류학에서 속의 개념이 어떻게 적용되는지를 보여줍니다.
c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d
0.953482645403377
한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다.

이 청크는 생물 분류학과 관련된 연구 문서에서 발췌되었습니다. 이 문서에서는 한 학생이 나무의 다양한 특성을 조사하는 과정에서 성장 속도, 온도 범위, 크기가 비슷하지만 잎과 꽃이 다른 두 나무를 발견한 사례를 설명하고 있습니다. 이 사례는 나무의 분류와

In [15]:
from typing import Optional
from pydantic import BaseModel, Field

class Search(BaseModel):
    """Search over a document database."""

    query: str = Field(
        ...,
        description="Similarity search query applied to document content.",
    )
    docid: Optional[str] = Field(None, description="Document ID for precise document retrieval.")
    content_snippet: Optional[str] = Field(None, description="A snippet of the document content to search for.")

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of documents. Given a question, return a list of database queries optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [17]:
query_analyzer.invoke("복숭아 키우는 노하우")

Search(query='복숭아 키우는 노하우', docid=None, content_snippet=None)

In [14]:
from typing import List
from langchain_core.documents import Document

def retrieval(search: Search) -> List[Document]:
    if search.docid is not None:
        # 필터링을 위해 docid 사용
        _filter = {"docid": {"$eq": search.docid}}
    else:
        _filter = None
    return retriever.invoke(search.query, filter=_filter)

retrieval_chain = query_analyzer | retrieval

# 특정 질의를 수행하여 문서를 검색
results = retrieval_chain.invoke("에너지 균형에 대한 문서 검색")

# 결과에서 docid와 내용을 출력
[(doc.metadata["docid"], doc.metadata["score"], doc.page_content[:100]) for doc in results]


[('42508ee0-c543-4338-878e-d98c6babee66',
  1.0,
  '건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간'),
 ('42508ee0-c543-4338-878e-d98c6babee66',
  0.961166253101737,
  '에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를\n'),
 ('42508ee0-c543-4338-878e-d98c6babee66',
  0.9606894841269841,
  '에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지\n\n이'),
 ('42508ee0-c543-4338-878e-d98c6babee66',
  0.9533577533577533,
  '건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.\n\n이 청크는 건강한 생활습관과 관련된 영양학 문서'),
 ('42508ee0-c543-4338-878e-d98c6babee66',
  0.9471593644679827,
  '유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의\n\n이'),
 ('42508ee0-c543-4338-878e-d98c6babee66',
  0.9317863805970149,
  '합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할